In [2]:
import pandas as pd
from collections import Counter
import string
import jctconv
import emoji
import re

ModuleNotFoundError: No module named 'jctconv'

In [3]:
# data check
df=pd.read_csv("all_rakuma.csv")

In [4]:
df.head()

,date,item_url,title,price,brand,category,good,bad,normal,ship_days,comments,last_log_date,good_counts,prefecture,delivery_fee,seller_url,size,picture_url,description,details_dict
0,2019/10/08 23:39:51,https://item.fril.jp/055f4080a2a0ae275db6ba7be...,Epiphone LTD Korina Explorer Bass,40000,Epiphone,楽器 > ベース > エレキベース,109,0,0,1-2日後,NaN,2000-01-01,0,岐阜県,着払い,https://fril.jp/shop/42df208014145c438f96e01f5...,なし,https://img.fril.jp/img/259586636/l/734809353....,コリーナボディ ブラックピックガード ゴールドパーツの豪華なエクスプローラー コリーナ ...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', 'ブランド':..."
1,2019/10/08 23:39:51,https://item.fril.jp/896a1d0197d05cf8431c136d9...,TVB-100 (SUN),38000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,"['TOKYO GUITARS', '本モデルを含め、TOKYO GUITARS工房にて製造...",2019/08/31,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223532100/l/635334018....,Tokyo Guitars モデル名 TVB 100 SUN Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
2,2019/10/08 23:39:51,https://item.fril.jp/6bdb6b4dd0f6c518938b94ec2...,TRB-4000 (BL),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,NaN,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223294381/l/634674588....,Tokyo Guitars モデル名 TRB 4000 BL Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
3,2019/10/08 23:39:51,https://item.fril.jp/b89c82258515163e5aa36aa48...,TRB-4000 (FG),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,NaN,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223294160/l/634673947....,Tokyo Guitars モデル名 TRB 4000 FG Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
4,2019/10/08 23:39:51,https://item.fril.jp/cf904ef67c6cec09a7465e7f4...,TB-100 (BRWN),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,NaN,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/216145047/l/614817207....,Tokyo Guitars モデル名 TB 100 BRWN Tokyo Guit...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122070 entries, 0 to 122069
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   date           122070 non-null  object
 1   item_url       122070 non-null  object
 2   title          122070 non-null  object
 3   price          122070 non-null  int64 
 4   brand          122070 non-null  object
 5   category       122070 non-null  object
 6   good           122070 non-null  int64 
 7   bad            122070 non-null  int64 
 8   normal         122070 non-null  int64 
 9   ship_days      122070 non-null  object
 10  comments       11186 non-null   object
 11  last_log_date  122070 non-null  object
 12  good_counts    122070 non-null  int64 
 13  prefecture     122070 non-null  object
 14  delivery_fee   122070 non-null  object
 15  seller_url     122070 non-null  object
 16  size           122070 non-null  object
 17  picture_url    122070 non-null  object
 18  desc

In [6]:
# category number
df.category.nunique()
# check category percentage
df.category.value_counts() / len(df)*100

メンズ > 靴/シューズ > スニーカー                     9.174244
エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ        8.975178
エンタメ/ホビー > フィギュア > アニメ/ゲーム               5.418203
エンタメ/ホビー > CD > ポップス/ロック(洋楽)             2.919636
エンタメ/ホビー > フィギュア > 特撮                    2.577210
                                           ...   
スポーツ/アウトドア > スポーツ/アウトドアその他 > ボウリング       0.000819
スポーツ/アウトドア > スポーツ/アウトドアその他 > バスケットボール    0.000819
楽器 > DJ機器 > その他                          0.000819
ハンドメイド > フラワー/ガーデン > ドライフラワー             0.000819
コスメ/美容 > ヘアケア > 白髪染め                     0.000819
Name: category, Length: 1012, dtype: float64

In [7]:
ctg_df=df.category.value_counts().reset_index()
# 1000 = 700:for AI study. 300;for test.
over1000_data=ctg_df[ctg_df.category>1000]
over1000_data.category.sum()

63006

In [14]:
over1000_data[["index"]]

,index
0,メンズ > 靴/シューズ > スニーカー
1,エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ
2,エンタメ/ホビー > フィギュア > アニメ/ゲーム
3,エンタメ/ホビー > CD > ポップス/ロック(洋楽)
4,エンタメ/ホビー > フィギュア > 特撮
5,エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー
6,スポーツ/アウトドア > フィッシング > ルアー用品
7,エンタメ/ホビー > トレーディングカード > シングルカード
8,エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー
9,エンタメ/ホビー > CD > アニメ


In [20]:
data=df.merge(over1000_data[["index"]],left_on="category",right_on="index")

In [22]:
data.category.nunique()

22